# Exploratory Data Analysis with R and BigQuery
## Overview

This notebook illustrates how to perform Exploratory Data Analysis (EDA) using R, on data extracted from BigQuery. After analysing and processing the data, the transformed data is stored in Cloud Storage (GCS) for further Machine Learning (MLL) tasks.

R is one of the most widely used programming languages for statistical modeling, which has a large and active community of data scientists and ML professional. 
With over 10,000 packages in the open-source repository of CRAN, R caters to all statistical data analysis applications, ML, and visualisation.


## Dataset
The dataset used in this tutorial is natality data, which describes all United States births registered in the 50 States, the District of Columbia, and New York City from 1969 to 2008, with more than 137 million records.
The dataset is available in [BigQuery public dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table&_ga=2.99329886.-1705629017.1551465326&_gac=1.109796023.1561476396.CI2rz-z4hOMCFc6RhQods4oEXA).

In this notebook, we focus on Exploratory Data Analysis, while the goal is to predict the baby's weight given a number of factors about the pregnancy and the baby's mother.

## Objective
The goal of this tutorial is to:
1. Query and analyse data from BigQuery using bigrquery R library
2. Prepare and store data in Cloud Storage for ML 


## Costs
This tutorial uses billable components of Google Cloud Platform (GCP):
1. AI Platform Notebooks
2. BigQuery
3. Cloud Storage

Learn about GCP pricing, use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## 0. Setup 
Install and import the required libraries. 

This may take several minutes if not installed already...

In [ ]:
# install the required R libraries
install.packages(c("bigrquery", "ggplot2", "readr"))

In [ ]:
# import R libraries
library(bigrquery) # used for querying BigQuery
library(ggplot2) # used for visualisation
library(dplyr) # used for data wrangling
library(readr) # required by bigrquery

Set your `PROJECT_ID` and `BUCKET_NAME`

In [ ]:
# Provide authentication through the JSON service account key
set_service_token("service-account-key.json")

# Set the project id
PROJECT_ID <- "r-on-gcp"

# Set yout GCS bucket
BUCKET_NAME <- "r-on-gcp"

## 1. Querying Data from BigQuery 

### 1.1. Prepare the BigQuery query

In [ ]:
sql_query <- "
    SELECT
      ROUND(weight_pounds, 2) AS weight_pounds ,
      is_male,
      mother_age,
      mother_race,
      plurality,
      gestation_weeks,
      mother_married,
      cigarette_use,
      alcohol_use,
      CAST(ABS(FARM_FINGERPRINT(
        CONCAT(
        CAST(YEAR AS STRING), CAST(month AS STRING), CAST(weight_pounds AS STRING)))) AS STRING) AS key
    FROM
        publicdata.samples.natality
    WHERE 
      year > 2000
      AND weight_pounds > 0
      AND mother_age > 0
      AND plurality > 0
      AND gestation_weeks > 0
      AND month > 0
    LIMIT %s
"

### 1.2. Execute the query 
The data will be retreived from BigQuery, and the results will be stored in an in-memory dataframe.

In [ ]:
sample_size <- 10000
sql_query <- sprintf(sql_query, sample_size)

natality_data <- bq_table_download(
    bq_project_query(
        PROJECT_ID, 
        query=sql_query
    )
)

### 1.3. View the query results

In [ ]:
# View the query result
head(natality_data)

In [ ]:
# View the results summary
summary(natality_data)

# 2. Visualising retreived data

In [ ]:
# Display the distribution of the baby weights using a Histogram
ggplot(
    data=natality_data, 
    aes(x = weight_pounds)
) + geom_histogram(bins = 200)

In [ ]:
# Display the correlation between the gestations weeks and baby weights 
ggplot(
    data=natality_data, 
    aes(x = gestation_weeks, y = weight_pounds)
) + geom_point() + geom_smooth(method = "lm")

### Performing the processing in BigQuery
Create function that finds the number of records and the average weight for each value of the chosen column

In [ ]:
get_distinct_values <- function(column_name) {
    query <- paste0(
        'SELECT ', column_name, ', 
            COUNT(1) AS num_babies,
            AVG(weight_pounds) AS avg_wt
        FROM publicdata.samples.natality
        WHERE year > 2000
        GROUP BY ', column_name)
    query_exec(query, project = PROJECT_ID, useLegacySql = FALSE)
}

In [ ]:
df <- get_distinct_values('is_male')
ggplot(data = df, aes(x = is_male, y = num_babies)) + geom_bar(stat = "identity")
ggplot(data = df, aes(x = is_male, y = avg_wt)) + geom_bar(stat = "identity")

In [ ]:
df <- get_distinct_values('mother_age')
ggplot(data = df, aes(x = mother_age, y = num_babies)) + geom_line(stat = "identity")
ggplot(data = df, aes(x = mother_age, y = avg_wt)) + geom_line(stat = "identity")

In [ ]:
df <- get_distinct_values('plurality')
ggplot(data=df, aes(x=plurality, y=num_babies)) + geom_bar(stat="identity") + scale_y_log10()
ggplot(data=df, aes(x=plurality, y=avg_wt)) + geom_bar(stat = "identity")

In [ ]:
df <- get_distinct_values('gestation_weeks')
ggplot(data = df, aes(x = gestation_weeks, y = num_babies)) + geom_bar(stat = "identity") + scale_y_log10()
ggplot(data = df, aes(x = gestation_weeks, y = avg_wt)) + geom_bar(stat = "identity")

# 3. Saving the data as CSV to GCS

In [ ]:
# Prepare train and evaluation data from BigQuery
sample_size <- 10000
sql_query <- sprintf(sql_query, sample_size)

train_query <- paste('SELECT * FROM (', sql_query, ') WHERE MOD(CAST(key AS INT64), 100) <= 75')
eval_query <- paste('SELECT * FROM (', sql_query, ') WHERE MOD(CAST(key AS INT64), 100) > 75')

# Load train data to data.frame
train_data <- bq_table_download(
    bq_project_query(
        PROJECT_ID, 
        query=train_query
    )
)

# Load evaluation data to data.frame
eval_data <- bq_table_download(
    bq_project_query(
        PROJECT_ID, 
        query=eval_query
    )
)

In [ ]:
print(paste0("Training instances count: ", nrow(train_data)))
print(paste0("Evaluation instances count: ",nrow(eval_data)))

In [ ]:
# Write data frames to a local CSV files, without headers nor row index
dir.create(file.path('data'), showWarnings = FALSE)
write.table(train_data, "data/train_data.csv", row.names=FALSE, col.names=FALSE, sep=",")
write.table(eval_data, "data/eval_data.csv", row.names=FALSE, col.names=FALSE, sep=",")

In [ ]:
# Upload CSV data to GCS
gcs_url <- paste0("gs://", BUCKET_NAME, "/")
command <- paste("gsutil mb", gcs_url)
system(command)
gcs_data_dir <- paste0("gs://", BUCKET_NAME, "/data")
command <- paste("gsutil cp data/*_data.csv", gcs_data_dir)
system(command)

In [ ]:
command <- paste("gsutil ls -l", gcs_data_dir)
system(command, intern = TRUE)

## License

Authors: Daniel Sparing & Khalid Salama

---
**Disclaimer**: This is not an official Google product. The sample code provided for an educational purpose.

---

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.